In [4]:
import pandas as pd
import numpy as np
import matplotlib
import plotnine as ggplot
%matplotlib inline

import glob
import re

import time

from pandas import DataFrame
from IPython.display import HTML

import sys
sys.path.append("fastai/old/")
from fastai_v7 import structured

In [581]:
!ls /home/maghoi/bin/software/PDB_Tool

1pazA.txt   PDB_Tool	 test.acc	 test.cle	 test.pdb
examples    README	 test.acc_value  test.distances  test.sse
LICENCE.md  source_code  test.ami	 test.feature


- Want AMI, SSE, ACC and FEAT = 13

In [3]:
1+3+4+5

13

In [4]:
!/home/maghoi/bin/software/PDB_Tool/PDB_Tool

========================================================|
PDB_Tool  (version 4.80) [2018.05.20]                   |
          a versatile tool to process PDB file          |
Usage:   ./PDB_Tool <-i input> <-r range> <-o output>   |
Or,      ./PDB_Tool <-i inroot> <-L list> <-o outroot>  |
--------------------------------------------------------|
For <-i input> input type,                              |
-i input  : input original PDB file, e.g., 1col.pdb     |
-r range  : input residue range, e.g., A:1-51           |
-o output : output PDB file, e.g., 1colA.pdb            |
--------------------------------------------------------|
For <-L list> input type,                               |
-i inroot  : input directory of original PDBs           |
-o outroot : output directory for generated PDBs        |
The rows in <list> indicate [input range output]        |
          e.g.,  1col.pdb A:1-51 1colA.pdb              |
========================================================|
              

In [4]:
%%bash
ls -al /scratch/maghoi/pMHC_data/features
cd /scratch/maghoi/pMHC_data/features
#rm /scratch/maghoi/pMHC_data/features/*

total 478488
drwxr-xr-x 2 maghoi unixusers 430080 Mar 14 14:34 .
drwxr-xr-x 7 maghoi unixusers    164 Mar 21 13:09 ..
-rw-r--r-- 1 maghoi unixusers     37 Mar 14 14:13 1g6r_1p_N1_1t_80s_pMHC-TCR.ami
-rw-r--r-- 1 maghoi unixusers     37 Mar 14 14:13 1g6r_1p_N1_1t_80s_pMHC-TCR.cle
-rw-r--r-- 1 maghoi unixusers  45875 Mar 14 14:36 1g6r_1p_N1_1t_80s_pMHC-TCR.feature
-rw-r--r-- 1 maghoi unixusers 259686 Mar 14 14:36 1g6r_1p_N1_1t_80s_pMHC-TCR.pdb
-rw-r--r-- 1 maghoi unixusers   5751 Mar 14 14:13 1g6r_1p_N1_1t_80s_pMHC-TCR_pep.pdb
-rw-r--r-- 1 maghoi unixusers     37 Mar 14 14:13 1g6r_1p_N1_1t_80s_pMHC-TCR.sse
-rw-r--r-- 1 maghoi unixusers     37 Mar 14 14:13 1g6r_1p_N1_1t_90s_pMHC-TCR.ami
-rw-r--r-- 1 maghoi unixusers     37 Mar 14 14:13 1g6r_1p_N1_1t_90s_pMHC-TCR.cle
-rw-r--r-- 1 maghoi unixusers  45875 Mar 14 14:11 1g6r_1p_N1_1t_90s_pMHC-TCR.feature
-rw-r--r-- 1 maghoi unixusers 259686 Mar 14 14:11 1g6r_1p_N1_1t_90s_pMHC-TCR.pdb
-rw-r--r-- 1 maghoi unixusers   5751 Mar 14 14:13 1g6r_1p_N1

bash: module: line 1: syntax error: unexpected end of file
bash: error importing function definition for `module'


### PDB_Tool Feature extraction (complex + peptide)

In [6]:
start = time.time()

#### Run PDB_tool

In [163]:
%%bash
cd /scratch/maghoi/pMHC_data/model_structures
#Extracting features for full complex
find . -name "*pMHC-TCR.pdb" -exec /home/maghoi/bin/software/PDB_Tool/PDB_Tool -i {} -o /scratch/maghoi/pMHC_data/features/{} -F 4 -r ! \;

#Extracting peptide amino acid sequence
find . -name "*pMHC-TCR.pdb" -exec /home/maghoi/bin/software/PDB_Tool/PDB_Tool -i {} -o /scratch/maghoi/pMHC_data/features/{}_pep -F 1 -r P:@ \;


#### Check outdir

In [173]:
%%bash
ls /scratch/maghoi/pMHC_data/features/ |head

1g6r_1p_N1_1t_80s_pMHC-TCR.ami
1g6r_1p_N1_1t_80s_pMHC-TCR.cle
1g6r_1p_N1_1t_80s_pMHC-TCR.feature
1g6r_1p_N1_1t_80s_pMHC-TCR.pdb
1g6r_1p_N1_1t_80s_pMHC-TCR.sse
1g6r_1p_N1_1t_80s_pMHC-TCR_pep.pdb
1g6r_1p_N1_1t_90s_pMHC-TCR.ami
1g6r_1p_N1_1t_90s_pMHC-TCR.cle
1g6r_1p_N1_1t_90s_pMHC-TCR.feature
1g6r_1p_N1_1t_90s_pMHC-TCR.pdb


In [165]:
print("Time:", round(time.time() - start, 2), "s")

Time: 122.0 s


#### Renaming filename.pdb_pep to filename_pep.pdb

In [171]:
%%bash
cd /scratch/maghoi/pMHC_data/features/
find . -type f -name "*_pep" |
    while IFS= read file_name; do
        mv $file_name ${file_name:0:-8}_pep.pdb
    done

In [172]:
!ls /scratch/maghoi/pMHC_data/features/ |head

1g6r_1p_N1_1t_80s_pMHC-TCR.ami
1g6r_1p_N1_1t_80s_pMHC-TCR.cle
1g6r_1p_N1_1t_80s_pMHC-TCR.feature
1g6r_1p_N1_1t_80s_pMHC-TCR.pdb
1g6r_1p_N1_1t_80s_pMHC-TCR.sse
1g6r_1p_N1_1t_80s_pMHC-TCR_pep.pdb
1g6r_1p_N1_1t_90s_pMHC-TCR.ami
1g6r_1p_N1_1t_90s_pMHC-TCR.cle
1g6r_1p_N1_1t_90s_pMHC-TCR.feature
1g6r_1p_N1_1t_90s_pMHC-TCR.pdb
ls: write error: Broken pipe


#### Check that there is 1464 feature files and peptide files
#### (will extract pep seq using pdb2pep)

In [5]:
%%bash
cd /scratch/maghoi/pMHC_data/features/
ls *pMHC-TCR.feature*|wc
ls *pMHC-TCR_pep*|wc

   1464    1464   51240
   1464    1464   51240


bash: module: line 1: syntax error: unexpected end of file
bash: error importing function definition for `module'


### Convert feature files to tab-delimited, extract relevant columns

In [296]:
%%bash
cd /scratch/maghoi/pMHC_data/features/
mkdir -p /scratch/maghoi/pMHC_data/features2/
find . -type f -name "*pMHC-TCR.feature" |
    while IFS= read file_name; do
        #cat $file_name
        sed -r 's/ +/\t/g' $file_name | cut -f 2,3,5,6,7,8,9,10 > ../features2/$file_name
    done

In [301]:
%%bash
ls /scratch/maghoi/pMHC_data/features2/|wc
head /scratch/maghoi/pMHC_data/features2/1g6r_1p_N1_1t_80s_pMHC-TCR.feature

   1464    1464   51240
Num	Res	SSE	CLE	ACC	pACC	CNa	CNb
1	G	L	R	2	99	2	2
2	P	L	R	2	70	4	5
3	H	E	E	1	29	6	8
4	S	E	F	1	12	7	9
5	L	E	D	0	0	9	9
6	R	E	E	1	26	8	7
7	Y	E	E	0	0	8	10
8	F	E	E	1	31	7	8
9	V	E	E	0	2	7	13


In [9]:
!ls -al /scratch/maghoi/pMHC_data/features2/

total 11808
drwxr-xr-x 2 maghoi unixusers 73728 Mar 14 14:46 .
drwxr-xr-x 7 maghoi unixusers   164 Mar 21 13:09 ..
-rw-r--r-- 1 maghoi unixusers  7567 Mar 14 14:48 1g6r_1p_N1_1t_80s_pMHC-TCR.feature
-rw-r--r-- 1 maghoi unixusers  7559 Mar 14 14:48 1g6r_1p_N1_1t_90s_pMHC-TCR.feature
-rw-r--r-- 1 maghoi unixusers  7564 Mar 14 14:48 1g6r_1p_N1_1t_95s_pMHC-TCR.feature
-rw-r--r-- 1 maghoi unixusers  7532 Mar 14 14:48 1g6r_1p_N1_1t_99s_pMHC-TCR.feature
-rw-r--r-- 1 maghoi unixusers  7566 Mar 14 14:48 1g6r_1p_N2_1t_80s_pMHC-TCR.feature
-rw-r--r-- 1 maghoi unixusers  7563 Mar 14 14:48 1g6r_1p_N2_1t_90s_pMHC-TCR.feature
-rw-r--r-- 1 maghoi unixusers  7566 Mar 14 14:48 1g6r_1p_N2_1t_95s_pMHC-TCR.feature
-rw-r--r-- 1 maghoi unixusers  7530 Mar 14 14:48 1g6r_1p_N2_1t_99s_pMHC-TCR.feature
-rw-r--r-- 1 maghoi unixusers  7576 Mar 14 14:48 1g6r_1p_N3_1t_80s_pMHC-TCR.feature
-rw-r--r-- 1 maghoi unixusers  7565 Mar 14 14:48 1g6r_1p_N3_1t_90s_pMHC-TCR.feature
-rw-r--r-- 1 maghoi unixusers  7556 Mar 14 14

In [13]:
pd.read_csv("/scratch/maghoi/pMHC_data/features2/1g6r_1p_N1_1t_80s_pMHC-TCR.feature", delimiter="\t")[180:190]


Num Res SSE CLE  ACC  pACC  CNa  CNb
180  181   N   L   R    0     0    5   10
181  182   I   L   C    0     0    3    7
182  183   R   L   C    0     2    7    7
183  184   Y   L   C    0     0    4    6
184  185   V   L   D    0     1    5    8
185  186   K   L   C    0     1    7   11
186  187   V   L   R    0     3    6    7
187  188   Q   L   R    1    31    6    9
188  189   S   E   R    2    53    6    4
189  190   V   E   E    0    10   11   11

In [12]:
pd.read_csv("/scratch/maghoi/pMHC_data/features2/2oi9_3p_S1_1t_99s_pMHC-TCR.feature", delimiter="\t")[180:190]


Num Res SSE CLE  ACC  pACC  CNa  CNb
180  181   P   L   C    0     0    1    2
181  182   F   L   C    0     1    4    7
182  183   D   L   C    0     8    7    9
183  184   L   L   R    0     1    4    8
184  185   L   L   R    2    52    4    6
185  186   S   E   R    2    67    4    3
186  187   V   E   E    1    11    8    8
187  188   R   E   D    2    69    4    4
188  189   E   T   C    2    43    7    6
189  190   G   T   N    2    53    4    4

### Extract peptide seqs

In [11]:
%%bash
cd /scratch/maghoi/pMHC_data/features/
find . -type f -name "*pMHC-TCR_pep.pdb" |
    while IFS= read file_name; do
        /home/maghoi/bin/software/pdb2fasta/pdb2fasta.sh $file_name > ../pep/$file_name.fasta
    done

In [10]:
!cat /scratch/maghoi/pMHC_data/pep/1g6r_1p_N1_1t_90s_pMHC-TCR_pep.pdb.fasta

>./1g6r_1p_N1_1t_90s_pMHC-TCR_pep | chain P | 8 aa
KNIRYVKV


### Save to csv

In [ ]:
import glob
filelist = glob.glob("/scratch/maghoi/pMHC_data/features2/*")

In [1]:
%timeit
import os
for i in range(0, len(filelist)):
    df_raw = pd.read_csv(filelist[i], delimiter = "\t")
    #df = structured.proc_df(df_raw); df = df[0]
    df = df_raw
    filename = os.path.basename(filelist[i])
    df.to_csv("/scratch/maghoi/pMHC_data/features3/" + filename + ".csv")
    print(filename)
    print(df.shape)

NameError: name 'filelist' is not defined

### Data generator (use this 27/03/19)

In [6]:
#!ls /scratch/maghoi/pMHC_data/features3/
!cat /scratch/maghoi/pMHC_data/features3/3vxm_3p_P1_1t_80s_pMHC-TCR.feature.csv

,Num,Res,SSE,CLE,ACC,pACC,CNa,CNb
0,1,G,L,R,2,100,2,2
1,2,S,L,R,2,63,5,5
2,3,H,E,F,1,22,6,9
3,4,S,E,F,1,19,7,8
4,5,M,E,D,0,0,9,9
5,6,R,E,E,1,17,8,6
6,7,Y,E,E,0,0,9,10
7,8,F,E,E,1,42,7,7
8,9,S,E,E,0,1,7,11
9,10,T,E,E,1,34,7,6
10,11,S,E,E,0,0,10,11
11,12,V,E,E,1,35,7,5
12,13,S,E,E,1,15,7,6
13,14,R,E,E,1,25,6,5
14,15,P,T,C,1,31,4,4
15,16,G,T,N,2,100,0,0
16,17,R,T,M,2,76,0,0
17,18,G,E,C,2,64,1,1
18,19,E,E,Q,2,55,3,3
19,20,P,E,C,0,9,5,8
20,21,R,E,C,1,31,7,6
21,22,F,E,E,0,0,9,10
22,23,I,E,E,1,41,7,6
23,24,A,E,E,0,0,8,10
24,25,V,E,E,1,14,8,8
25,26,G,E,D,0,0,8,9
26,27,Y,E,E,1,31,7,8
27,28,V,E,B,0,0,8,8
28,29,D,T,N,1,29,5,3
29,30,D,T,N,2,99,0,0
30,31,T,E,G,1,30,3,5
31,32,Q,E,E,1,23,6,5
32,33,F,E,B,0,0,10,9
33,34,V,E,L,0,0,9,8
34,35,R,E,E,1,36,8,7
35,36,F,E,E,0,1,10,12
36,37,D,E,E,1,18,6,3
37,38,S,S,P,1,14,5,6
38,39,D,S,H,2,48,3,1
39,40,A,S,G,1,30,1,2
40,41,A,S,A,2,100,0,0
41,42,S,S,J,2,47,0,1
42,43,Q,L,O,1,29,5,2
43,44,R,S,G,2,43,4,3
44,45,M,L,C,0,3,8,10
45,46,E,E,F,1,25,4,5
46,47,P,E,C,1,43,5,

In [7]:
#Start position 0
def data_generator(ix_train = 1080, ix_val = 256, ix_test = 128, data_size=1464):
        filelist = glob.glob("/scratch/maghoi/pMHC_data/features3/*.csv")
        
        print(len(filelist))
        
        for i in range(len(filelist)):
            r = re.compile(r'.*/features3/(.*).feature')
            m = r.match(filelist[i])
            filename = m.group(1)
            filenames.append(filename)
        
        #Load data into dfs
        filelist_loaded = []
        for i in range(0, len(filelist)):
            df = pd.read_csv(filelist[i])
            #df = df.drop(["Unnamed: 0", "Num", "pACC", "CNb"], axis = 1)
            df = df.drop(["Unnamed: 0", "Num"], axis = 1)
            
            #Convert categorical values (res + solvent acc cateogry) to string
            filelist_loaded.append(df)
        return filelist_loaded, filenames

- Load feature files into filelist_loded and filenames

In [8]:
filelist_loaded = []
filenames = []
filelist_loaded, filenames = data_generator()

1464


- Extract aligned sequences from All_identity_xlsx
- Fix names so they are identical

In [9]:
aligned = pd.read_excel("/scratch/maghoi/pMHC_data/All_identity.xlsx")
aligned = aligned.iloc[:, [0, 3]]
aligned.Name = aligned.Name + "_pMHC-TCR"

In [10]:
aligned.loc[aligned.Name == filenames[2]].Aligned_sequence

48    GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...
Name: Aligned_sequence, dtype: object

In [11]:
for i in range(len(filenames)):
    print(aligned.loc[aligned.Name == filenames[i]])
    print("Items:", i)

                          Name  \
19  1g6r_1p_N1_1t_80s_pMHC-TCR   

                                     Aligned_sequence  
19  GPHSLRYFVTAVSRPGLGEPRYMEVGYVDDTEFVRFDSDAENPRYE...  
Items: 0
                           Name  \
660  3tjh_4p_P1_1t_90s_pMHC-TCR   

                                      Aligned_sequence  
660  GPHSMRYYETATSRRGLGEPRYTSVGYVDDKEFVRFDSDAENPRYE...  
Items: 1
                          Name  \
48  2bnq_2p_P1_1t_99s_pMHC-TCR   

                                     Aligned_sequence  
48  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
Items: 2
                          Name  \
13  1g6r_1p_N1_1t_90s_pMHC-TCR   

                                     Aligned_sequence  
13  GPHSLRYFVTAVSRPGLGEPRYMEVGYVDDTEFVRFDSDAENPRYE...  
Items: 3
                         Name  \
7  1g6r_1p_N1_1t_95s_pMHC-TCR   

                                    Aligned_sequence  
7  GPHSLRYFVTAVSRPGLGEPRYMEVGYVDDTEFVRFDSDAENPRYE...  
Items: 4
                          Name  \
27  1mwa_1p_N3_1

                           Name  \
102  2e7l_3p_P1_1t_95s_pMHC-TCR   

                                      Aligned_sequence  
102  GPHSMRYFETAVSRRGLGEPRYISVGYVDDKEFVRFDSDAENPRYE...  
Items: 79
                          Name  \
96  2e7l_3p_P1_1t_99s_pMHC-TCR   

                                     Aligned_sequence  
96  GPHSMRYFETAVSRRGLGEPRYISVGYVDDKEFVRFDSDAENPRYE...  
Items: 80
                           Name  \
118  2e7l_3p_R1_1t_80s_pMHC-TCR   

                                      Aligned_sequence  
118  GPHSMRYFETAVSRRGLGEPRYISVGYVDDKEFVRFDSDAENPRYE...  
Items: 81
                           Name  \
112  2e7l_3p_R1_1t_90s_pMHC-TCR   

                                      Aligned_sequence  
112  GPHSMRYFETAVSRRGLGEPRYISVGYVDDKEFVRFDSDAENPRYE...  
Items: 82
                           Name  \
106  2e7l_3p_R1_1t_95s_pMHC-TCR   

                                      Aligned_sequence  
106  GPHSMRYFETAVSRRGLGEPRYISVGYVDDKEFVRFDSDAENPRYE...  
Items: 83
                           Na

                           Name  \
168  2nx5_2p_P1_1t_99s_pMHC-TCR   

                                      Aligned_sequence  
168  GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTE...  
Items: 152
                           Name  \
190  2nx5_2p_R1_1t_80s_pMHC-TCR   

                                      Aligned_sequence  
190  GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTE...  
Items: 153
                           Name  \
184  2nx5_2p_R1_1t_90s_pMHC-TCR   

                                      Aligned_sequence  
184  GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTE...  
Items: 154
                           Name  \
178  2nx5_2p_R1_1t_95s_pMHC-TCR   

                                      Aligned_sequence  
178  GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTE...  
Items: 155
                           Name  \
172  2nx5_2p_R1_1t_99s_pMHC-TCR   

                                      Aligned_sequence  
172  GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTE...  
Items: 156
                    

                           Name  \
263  2p5w_2p_S1_1t_80s_pMHC-TCR   

                                      Aligned_sequence  
263  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
Items: 229
                           Name  \
257  2p5w_2p_S1_1t_90s_pMHC-TCR   

                                      Aligned_sequence  
257  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
Items: 230
                           Name  \
251  2p5w_2p_S1_1t_95s_pMHC-TCR   

                                      Aligned_sequence  
251  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
Items: 231
                           Name  \
245  2p5w_2p_S1_1t_99s_pMHC-TCR   

                                      Aligned_sequence  
245  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
Items: 232
                           Name  \
283  2pye_2p_N1_1t_80s_pMHC-TCR   

                                      Aligned_sequence  
283  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
Items: 233
                    

                           Name  \
317  2vlr_4p_S1_1t_99s_pMHC-TCR   

                                      Aligned_sequence  
317  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
Items: 304
                           Name  \
355  2ypl_3p_N1_1t_80s_pMHC-TCR   

                                      Aligned_sequence  
355  GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRMA...  
Items: 305
                           Name  \
349  2ypl_3p_N1_1t_90s_pMHC-TCR   

                                      Aligned_sequence  
349  GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRMA...  
Items: 306
                           Name  \
343  2ypl_3p_N1_1t_95s_pMHC-TCR   

                                      Aligned_sequence  
343  GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRMA...  
Items: 307
                           Name  \
337  2ypl_3p_N1_1t_99s_pMHC-TCR   

                                      Aligned_sequence  
337  GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRMA...  
Items: 308
                    

...

                            Name  \
1268  5c08_1p_N2_1t_80s_pMHC-TCR   

                                       Aligned_sequence  
1268  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
Items: 1148
                            Name  \
1262  5c08_1p_N2_1t_90s_pMHC-TCR   

                                       Aligned_sequence  
1262  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
Items: 1149
                            Name  \
1256  5c08_1p_N2_1t_95s_pMHC-TCR   

                                       Aligned_sequence  
1256  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
Items: 1150
                            Name  \
1250  5c08_1p_N2_1t_99s_pMHC-TCR   

                                       Aligned_sequence  
1250  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
Items: 1151
                            Name  \
1269  5c08_1p_N3_1t_80s_pMHC-TCR   

                                       Aligned_sequence  
1269  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
Items: 

                            Name  \
1179  5c0b_1p_N3_1t_99s_pMHC-TCR   

                                       Aligned_sequence  
1179  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
Items: 1227
                            Name  \
1194  5c0b_1p_P1_1t_80s_pMHC-TCR   

                                       Aligned_sequence  
1194  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
Items: 1228
                            Name  \
1188  5c0b_1p_P1_1t_90s_pMHC-TCR   

                                       Aligned_sequence  
1188  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
Items: 1229
                            Name  \
1182  5c0b_1p_P1_1t_95s_pMHC-TCR   

                                       Aligned_sequence  
1182  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
Items: 1230
                            Name  \
1176  5c0b_1p_P1_1t_99s_pMHC-TCR   

                                       Aligned_sequence  
1176  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
Items: 

                            Name  \
1342  5e9d_0p_R1_1t_80s_pMHC-TCR   

                                       Aligned_sequence  
1342  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
Items: 1304
                            Name  \
1336  5e9d_0p_R1_1t_90s_pMHC-TCR   

                                       Aligned_sequence  
1336  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
Items: 1305
                            Name  \
1330  5e9d_0p_R1_1t_95s_pMHC-TCR   

                                       Aligned_sequence  
1330  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
Items: 1306
                            Name  \
1324  5e9d_0p_R1_1t_99s_pMHC-TCR   

                                       Aligned_sequence  
1324  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
Items: 1307
                            Name  \
1343  5e9d_0p_S1_1t_80s_pMHC-TCR   

                                       Aligned_sequence  
1343  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
Items: 

                            Name  \
1402  5hhm_4p_R1_1t_95s_pMHC-TCR   

                                       Aligned_sequence  
1402  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
Items: 1378
                            Name  \
1396  5hhm_4p_R1_1t_99s_pMHC-TCR   

                                       Aligned_sequence  
1396  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
Items: 1379
                            Name  \
1415  5hhm_4p_S1_1t_80s_pMHC-TCR   

                                       Aligned_sequence  
1415  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
Items: 1380
                            Name  \
1409  5hhm_4p_S1_1t_90s_pMHC-TCR   

                                       Aligned_sequence  
1409  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
Items: 1381
                            Name  \
1403  5hhm_4p_S1_1t_95s_pMHC-TCR   

                                       Aligned_sequence  
1403  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
Items: 

                          Name  \
37  1mwa_1p_N1_1t_90s_pMHC-TCR   

                                     Aligned_sequence  
37  GPHSLRYFVTAVSRPGLGEPRYMEVGYVDDTEFVRFDSDAENPRYE...  
Items: 1454
                          Name  \
31  1mwa_1p_N1_1t_95s_pMHC-TCR   

                                     Aligned_sequence  
31  GPHSLRYFVTAVSRPGLGEPRYMEVGYVDDTEFVRFDSDAENPRYE...  
Items: 1455
                          Name  \
25  1mwa_1p_N1_1t_99s_pMHC-TCR   

                                     Aligned_sequence  
25  GPHSLRYFVTAVSRPGLGEPRYMEVGYVDDTEFVRFDSDAENPRYE...  
Items: 1456
                          Name  \
44  1mwa_1p_N2_1t_80s_pMHC-TCR   

                                     Aligned_sequence  
44  GPHSLRYFVTAVSRPGLGEPRYMEVGYVDDTEFVRFDSDAENPRYE...  
Items: 1457
                          Name  \
38  1mwa_1p_N2_1t_90s_pMHC-TCR   

                                     Aligned_sequence  
38  GPHSLRYFVTAVSRPGLGEPRYMEVGYVDDTEFVRFDSDAENPRYE...  
Items: 1458
                          Name  \
3

Check that filenames and aligned names match

In [18]:
len(filenames), len(aligned)

(1464, 1464)

### Align raw sequences to pre-aligned sequences
### Extract features from .feature
### Add new features for MHC, pep, TCR
- Pre-aligned seqs in All_identity.xlsx
- Residue to residue comparison, replace missing (padded) with -1
- Add features for corresponding positions, with -1 for missing (padded) positions
- Add to dfs (list of dataframes)

In [12]:
filelist_loaded[0].iloc[:]

Res SSE CLE  ACC  pACC  CNa  CNb
0     G   L   R    2    99    2    2
1     P   L   R    2    70    4    5
2     H   E   E    1    29    6    8
3     S   E   F    1    12    7    9
4     L   E   D    0     0    9    9
5     R   E   E    1    26    8    7
6     Y   E   E    0     0    8   10
7     F   E   E    1    31    7    8
8     V   E   E    0     2    7   13
9     T   E   E    1    34    8    6
10    A   E   E    0     6   10   10
11    V   E   E    1    22    7    6
12    S   E   D    0     6    8    6
13    R   E   E    1    29    7    5
14    P   T   E    1    29    6    4
15    G   T   N    2   100    1    1
16    L   T   J    2    74    0    0
17    G   E   L    2    53    1    1
18    E   E   Q    2    48    3    3
19    P   E   C    1    20    6    9
20    R   E   C    1    30    8    7
21    Y   E   E    0     1   10   11
22    M   E   E    1    23    7    8
23    E   E   E    0     0    8    9
24    V   E   E    1    14    8    8
25    G   E   E    0     0    9   10
26    Y   E   E    1    24    7    8
27    V   E   B    0     0    9    8
28    D   T   N    1    22    5    3
29    D   T   N    2    90    0    0
..   ..  ..  ..  ...   ...  ...  ...
375   A   L   C    0     0    8   10
376   T   L   E    2    45    5    5
377   P   G   A    2    53    4    3
378   S   G   J    2    68    1    0
379   Q   G   J    0     4    4    5
380   T   L   G    1    27    6    3
381   S   E   L    0     3    7    6
382   V   E   C    1    16    8    8
383   Y   E   E    0     0    8   12
384   F   E   E    0     0   11   12
385   C   E   E    0     0   11   14
386   A   E   E    0     0    9   10
387   S   E   E    0     1   10    9
388   G   E   F    0     0    9   11
389   G   S   D    1    27   11   11
390   G   S   N    1    18   12   11
391   G   S   K    1    16    8    9
392   T   L   L    1    10    3    7
393   L   E   D    0     1    5    7
394   Y   E   E    2    55    6    7
395   F   E   F    0     8    8   10
396   G   L   D    0     5    9    9
397   A   L   A    2    50    9    6
398   G   L   L    0     3    9    9
399   T   E   D    0     1   10   11
400   R   E   E    1    37    8    6
401   L   E   E    0     0    9   14
402   S   E   E    1    23    7    4
403   V   E   E    0     5   10   11
404   L   L   R    2    75    6    5

[405 rows x 7 columns]

In [20]:
len(aligned.loc[aligned.Name == filenames[x]].Aligned_sequence.values[0])

NameError: name 'x' is not defined

In [13]:
start = time.time()
dfs = []

for x in range(len(filenames)):
    #Sequences
    aligned_seq = aligned.loc[aligned.Name == filenames[x]].Aligned_sequence.values[0]
    raw_seq = filelist_loaded[x].iloc[:, 0].values
    
    #Raw features
    raw_1 = filelist_loaded[x].iloc[:, 1].values
    raw_2 = filelist_loaded[x].iloc[:, 2].values
    raw_3 = filelist_loaded[x].iloc[:, 3].values
    raw_4 = filelist_loaded[x].iloc[:, 4].values
    raw_5 = filelist_loaded[x].iloc[:, 5].values
    raw_6 = filelist_loaded[x].iloc[:, 6].values

    #New features
    new_seq = []
    feature1 = []
    feature2 = []
    feature3 = []
    feature4 = []
    feature5 = []
    feature6 = []

    i2 = 0
    for i in range(len(aligned_seq)):
        #print(i, aligned_seq[i], ":", i2, raw_seq[i2])
        #print(i, i2, len(raw_seq))
        if aligned_seq[i] == raw_seq[i2]:

            #Append values
            new_seq.append(raw_seq[i2])
            feature1.append(raw_1[i2])
            feature2.append(raw_2[i2])
            feature3.append(raw_3[i2])
            feature4.append(raw_4[i2])
            feature5.append(raw_5[i2])
            feature6.append(raw_6[i2])
            
            i2 += 1
            if i2 >= len(raw_seq):
                break
            #print("Appended")

        else:
            #Append 0
            new_seq.append("-1")
            feature1.append("-1")
            feature2.append("-1")
            feature3.append("-1")
            feature4.append("-1")
            feature5.append("-1")
            feature6.append("-1")
            
    #Final check
    while len(new_seq) < len(aligned_seq):
        new_seq.append("-1")
        feature1.append("-1")
        feature2.append("-1")
        feature3.append("-1")
        feature4.append("-1")
        feature5.append("-1")
        feature6.append("-1")            

            
    #Save to df and append to master df list
    df = pd.DataFrame({"Sequence": new_seq, "SSE": feature1, "CLE": feature2, "ACC": feature3, "pACC": feature4, "CNa": feature5, "CNb": feature6})
    df.index.name = filenames[x]
    dfs.append(df)
    #print("\n\n", new_seq)
    #print("Lengths:", len(aligned_seq), len("".join(raw_seq)), len("".join(new_seq)))

print("Time:", time.time() - start)

Time: 4.343043327331543


In [14]:
len(dfs)

1464

### Append MHC, pep, TCR a/b type per position

#### Structure aligned length calculations:

MHC: (1-181)

GPHSLRYFVTAVSRPGLGEPRYMEVGYVDDTEFVRFDSDAENPRYEPRARWMEQEG-PEYWERETQKAKGNEQSFRVSLRTLLGYYNQSAGGSHTIQVISGCEVGSDGRLLRGYQQYAYDGCDYIALNEDLKTWTAADMAALITKHKWEQAGEAER-LRAYLEGTCVEWLRRYLKNGNATL

Peptide: (1-11)

RAYGKNSLXXX

TCR: (1-276)

-QSVTQPDARVTVSEGASLQLRCKYSYSATP-------YLFWYVQY-PRQGLQLLLKYYSGD----PVVQ-GVNGFEAEF-S-KSNSSFHLRKASVHWSDSAVYFCAVSGFA-----------SALTFGSGTKVIVLP-AVTQSPRNKVAVTGGKVTLSCNQTNNHN------NMYWYRQDTGHGLRLIHYSYGAGS----TEKG--DIPD-GYKASRPSQENFSLILELATPSQTSVYFCASGGGGT----------------LYFGAGTRLSVL

TCR-A (1-138)

-QSVTQPDARVTVSEGASLQLRCKYSYSATP-------YLFWYVQY-PRQGLQLLLKYYSGD----PVVQ-GVNGFEAEF-S-KSNSSFHLRKASVHWSDSAVYFCAVSGFA-----------SALTFGSGTKVIVLP

TCR-B (1-38)

-AVTQSPRNKVAVTGGKVTLSCNQTNNHN------NMYWYRQDTGHGLRLIHYSYGAGS----TEKG--DIPD-GYKASRPSQENFSLILELATPSQTSVYFCASGGGGT----------------LYFGAGTRLSVL

1-468

0-467 (len(468))

In [16]:
### %time
# Pos 0 - 181 = MHC
# Pos 181 - 192 = Pep
# 192 - 330 = tcr a
# 330 - 468 = tcr b

#Create column of sequence type
dfs[0]["Type"] = 0
dfs[0]["Type"].iloc[0:181] = 1
dfs[0]["Type"].iloc[181:192] = 2
dfs[0]["Type"].iloc[192:330] = 3
dfs[0]["Type"].iloc[330:468] = 4

for i in range(1, len(dfs)):
    dfs[i]["Type"] = dfs[0]["Type"]

/home/maghoi/.conda/envs/fastai_py37/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


### Validate data

In [17]:
dfs[200]

Sequence SSE CLE ACC pACC CNa CNb  Type
2p5e_2p_P1_1t_99s_pMHC-TCR                                        
0                                 G   L   R   2  100   2   2     1
1                                 S   L   R   2   69   6   5     1
2                                 H   E   F   1   15   7   9     1
3                                 S   E   F   1   19   7   8     1
4                                 M   E   D   0    0   9   9     1
5                                 R   E   E   1   19   8   6     1
6                                 Y   E   E   0    0   8  10     1
7                                 F   E   E   1   41   7   7     1
8                                 F   E   E   0    2   7   9     1
9                                 T   E   E   1   26   8   7     1
10                                S   E   E   0    2  10  11     1
11                                V   E   E   1   34   7   5     1
12                                S   L   C   1   16   7   7     1
13                                R   L   E   1   23   7   5     1
14                                P   T   E   2   45   4   4     1
15                                G   T   N   2  100   0   0     1
16                                R   T   M   2   77   0   0     1
17                                G   S   C   2   66   1   1     1
18                                E   S   Q   1   24   3   4     1
19                                P   L   C   1   13   5   8     1
20                                R   E   C   1   37   8   7     1
21                                F   E   E   0    2   9  10     1
22                                I   E   E   1   30   7   6     1
23                                A   E   E   0    2   8  12     1
24                                V   E   E   1   16   8   7     1
25                                G   E   E   0    0   8   9     1
26                                Y   E   E   1   31   7   8     1
27                                V   E   B   0    0   9   9     1
28                                D   T   N   1   33   5   3     1
29                                D   T   N   2   83   0   0     1
...                             ...  ..  ..  ..  ...  ..  ..   ...
438                               G   S   L   0    0   7   9     4
439                               N   T   Q   1   30  10   5     4
440                               T   T   N   1   28   5   6     4
441                               G   L   O   0    4   5   7     4
442                               E   L   G   1   34   5   6     4
443                              -1  -1  -1  -1   -1  -1  -1     4
444                              -1  -1  -1  -1   -1  -1  -1     4
445                              -1  -1  -1  -1   -1  -1  -1     4
446                              -1  -1  -1  -1   -1  -1  -1     4
447                              -1  -1  -1  -1   -1  -1  -1     4
448                              -1  -1  -1  -1   -1  -1  -1     4
449                              -1  -1  -1  -1   -1  -1  -1     4
450                              -1  -1  -1  -1   -1  -1  -1     4
451                              -1  -1  -1  -1   -1  -1  -1     4
452                              -1  -1  -1  -1   -1  -1  -1     4
453                              -1  -1  -1  -1   -1  -1  -1     4
454                              -1  -1  -1  -1   -1  -1  -1     4
455                              -1  -1  -1  -1   -1  -1  -1     4
456                               L   L   C   0    3   4   9     4
457                               F   E   E   1   29   6   9     4
458                               F   E   E   1   14   8   8     4
459                               G   L   C   0    8   9   9     4
460                               E   L   A   2   62   7   4     4
461                               G   L   L   0    0   8   7     4
462                               S   E   D   0    0   9  10     4
463                               R   E   E   1   21   7   5     4
464                               L   E   E   0    1   9  12     4
465    

In [18]:
filelist_loaded[200]

Res SSE CLE  ACC  pACC  CNa  CNb
0     G   L   R    2   100    2    2
1     S   L   R    2    69    6    5
2     H   E   F    1    15    7    9
3     S   E   F    1    19    7    8
4     M   E   D    0     0    9    9
5     R   E   E    1    19    8    6
6     Y   E   E    0     0    8   10
7     F   E   E    1    41    7    7
8     F   E   E    0     2    7    9
9     T   E   E    1    26    8    7
10    S   E   E    0     2   10   11
11    V   E   E    1    34    7    5
12    S   L   C    1    16    7    7
13    R   L   E    1    23    7    5
14    P   T   E    2    45    4    4
15    G   T   N    2   100    0    0
16    R   T   M    2    77    0    0
17    G   S   C    2    66    1    1
18    E   S   Q    1    24    3    4
19    P   L   C    1    13    5    8
20    R   E   C    1    37    8    7
21    F   E   E    0     2    9   10
22    I   E   E    1    30    7    6
23    A   E   E    0     2    8   12
24    V   E   E    1    16    8    7
25    G   E   E    0     0    8    9
26    Y   E   E    1    31    7    8
27    V   E   B    0     0    9    9
28    D   T   N    1    33    5    3
29    D   T   N    2    83    0    0
..   ..  ..  ..  ...   ...  ...  ...
382   S   G   J    2    85    1    1
383   Q   G   J    0     7    4    6
384   T   L   M    1    34    6    3
385   S   E   L    0     7    7    4
386   V   E   C    1    18    8   10
387   Y   E   E    0     0    8   11
388   F   E   E    0     0    9   12
389   C   E   E    0     0   10   14
390   A   E   E    0     0    9    8
391   S   E   E    0     0   10   11
392   S   E   E    0     0    8    6
393   Y   L   C    1    22   10   10
394   L   S   A    0     1   11   11
395   G   S   L    0     0    7    9
396   N   T   Q    1    30   10    5
397   T   T   N    1    28    5    6
398   G   L   O    0     4    5    7
399   E   L   G    1    34    5    6
400   L   L   C    0     3    4    9
401   F   E   E    1    29    6    9
402   F   E   E    1    14    8    8
403   G   L   C    0     8    9    9
404   E   L   A    2    62    7    4
405   G   L   L    0     0    8    7
406   S   E   D    0     0    9   10
407   R   E   E    1    21    7    5
408   L   E   E    0     1    9   12
409   T   E   E    2    51    7    4
410   V   E   E    0     4   10   11
411   L   L   R    2    68    6    5

[412 rows x 7 columns]

### Save data

In [104]:
!mkdir -p /scratch/maghoi/pMHC_data/features5/
!rm /scratch/maghoi/pMHC_data/features5/*
!ls /scratch/maghoi/pMHC_data/features5/

In [19]:
for i in range(len(dfs)):
    dfs[i].to_csv("/scratch/maghoi/pMHC_data/features5/" + dfs[i].index.name + ".csv")

In [20]:
!ls /scratch/maghoi/pMHC_data/features5/|wc

   1464    1464   45384


In [21]:
len(filenames)

1464

### Load data for pre-proc

In [22]:
import sys
sys.path.append("fastai/old/")
from fastai_v7.structured import *
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer, OneHotEncoder
import os
import re
import glob
#os.environ["CUDA_VISIBLE_DEVICES"]="7"

from pandas import DataFrame
from IPython.display import HTML

import warnings
warnings.filterwarnings(action='once')
warnings.simplefilter('ignore')


In [24]:
filelist_loaded = []
filelist = glob.glob("/scratch/maghoi/pMHC_data/features5/*.csv"); len(filelist)
for i in range(0, len(filelist)):
    df = pd.read_csv(filelist[i])
    #df = df.drop(["Unnamed: 0", "Num", "pACC", "CNb"], axis = 1)

    #Convert categorical values (res + solvent acc cateogry) to string
    #df.iloc[:, 0:2] = df.iloc[:, 0:2].astype(int).astype(str)
    #print(df.shape)
    filelist_loaded.append(df)

### 1-hot dummy rows

In [25]:
dummy = pd.DataFrame({"Positon": np.tile(range(1,2), 21)[0:21],
            "Sequence": range(0,21),
            "SSE": np.tile(range(0,8), 6)[0:21],
            "CLE": np.tile(range(0,18), 6)[0:21],
            "ACC": np.tile(range(0,4), 6)[0:21],
            "pACC": np.tile(range(1,2), 21)[0:21],
            "CNa": np.tile(range(1,2), 21)[0:21],
            "CNb": np.tile(range(1,2), 21)[0:21],
            "Type": np.tile(range(1,5), 7)[0:21]})
dummy

Positon  Sequence  SSE  CLE  ACC  pACC  CNa  CNb  Type
0         1         0    0    0    0     1    1    1     1
1         1         1    1    1    1     1    1    1     2
2         1         2    2    2    2     1    1    1     3
3         1         3    3    3    3     1    1    1     4
4         1         4    4    4    0     1    1    1     1
5         1         5    5    5    1     1    1    1     2
6         1         6    6    6    2     1    1    1     3
7         1         7    7    7    3     1    1    1     4
8         1         8    0    8    0     1    1    1     1
9         1         9    1    9    1     1    1    1     2
10        1        10    2   10    2     1    1    1     3
11        1        11    3   11    3     1    1    1     4
12        1        12    4   12    0     1    1    1     1
13        1        13    5   13    1     1    1    1     2
14        1        14    6   14    2     1    1    1     3
15        1        15    7   15    3     1    1    1     4
16        1        16    0   16    0     1    1    1     1
17        1        17    1   17    1     1    1    1     2
18        1        18    2    0    2     1    1    1     3
19        1        19    3    1    3     1    1    1     4
20        1        20    4    2    0     1    1    1     1

### 1-hot, normalize

In [42]:
dfs = []
for i in range(len(filelist_loaded)):
    #Process categorical to values using fast.ai proc_df (only processes string values)
    df = proc_df(filelist_loaded[i])[0] 
    if len(df) != 468:
        print("Expected df length", 468, "found", len(df))
        break

    #Change proc_df´ed values to 0 = missing value, 1 as lowest real value
    #Change rest of features to 0 = missing value, 1 as lowest real value
    df.iloc[:, [1, 2, 3]] = df.iloc[:, [1, 2, 3]] - 1
    df.iloc[:, [0, 4, 5, 6, 7]] = df.iloc[:, [0, 4, 5, 6, 7]] + 1

    #Append dummy column
    df.columns = dummy.columns.values
    df = df.append(dummy)

    #One-hot encoding for Sequence, SSE, CLE, ACC, type
    onehotencoder = OneHotEncoder(categorical_features = [1, 2, 3, 4, 8])
    df = onehotencoder.fit_transform(df).toarray()
    
    #Remove 1-hot columns encoding missing positions where value = 1 (unnecessary)
    #Columns correspond to first columns for seq, SSE, CLE, ACC
    #Remove dummy columns
    df = np.delete(df, [0, 21, 29, 48], axis = 1)
    df = df[0:468]
    
    #Normalization for columns sequence position, pACC, CNa, CNb
    df[:, -4: ] = preprocessing.normalize(df[:, -4: ])
    
    dfs.append(df)
    
    if df.shape != (468, 56):
        print(i, "expected shape", (468, 56), "got", df.shape)
        break

In [34]:
HTML(DataFrame(dfs[0]).to_html())

In [38]:
#Data here shown WIHTOUT df = np.delete(df, [0, 21, 29, 48], axis = 1)
HTML(DataFrame(dfs[0][455]).to_html()) #Can remove columns 0, 21, 29, 48 

In [40]:
proc_df(filelist_loaded[0])[0]

1g6r_1p_N1_1t_80s_pMHC-TCR  Sequence  SSE  CLE  ACC  pACC  CNa  CNb  Type
0                             0         7    6   19    2    99    2    2     1
1                             1        14    6   19    2    70    4    5     1
2                             2         8    3    6    1    29    6    8     1
3                             3        17    3    7    1    12    7    9     1
4                             4        11    3    5    0     0    9    9     1
5                             5        16    3    6    1    26    8    7     1
6                             6        21    3    6    0     0    8   10     1
7                             7         6    3    6    1    31    7    8     1
8                             8        19    3    6    0     2    7   13     1
9                             9        18    3    6    1    34    8    6     1
10                           10         2    3    6    0     6   10   10     1
11                           11        19    3    6    1    22    7    6     1
12                           12        17    3    5    0     6    8    6     1
13                           13        16    3    6    1    29    7    5     1
14                           14        14    8    6    1    29    6    4     1
15                           15         7    8   15    2   100    1    1     1
16                           16        11    8   11    2    74    0    0     1
17                           17         7    3   13    2    53    1    1     1
18                           18         5    3   18    2    48    3    3     1
19                           19        14    3    4    1    20    6    9     1
20                           20        16    3    4    1    30    8    7     1
21                           21        21    3    6    0     1   10   11     1
22                           22        12    3    6    1    23    7    8     1
23                           23         5    3    6    0     0    8    9     1
24                           24        19    3    6    1    14    8    8     1
25                           25         7    3    6    0     0    9   10     1
26                           26        21    3    6    1    24    7    8     1
27                           27        19    3    3    0     0    9    8     1
28                           28         4    8   15    1    22    5    3     1
29                           29         4    8   15    2    90    0    0     1
..                          ...       ...  ...  ...  ...   ...  ...  ...   ...
438                         438         7    7   12    1    16    8    9     4
439                         439        18    6   13    1    10    3    7     4
440                         440         1    1    1   -1    -1   -1   -1     4
441                         441         1    1    1   -1    -1   -1   -1     4
442                         442         1    1    1   -1    -1   -1   -1     4
443                         443         1    1    1   -1    -1   -1   -1     4
444                         444         1    1    1   -1    -1   -1   -1     4
445                         445         1    1    1   -1    -1   -1   -1     4
446                         446         1    1    1   -1    -1   -1   -1     4
447                         447         1    1    1   -1    -1   -1   -1     4
448                         448         1    1    1   -1    -1   -1   -1     4
449                         449         1    1    1   -1    -1   -1   -1     4
450                         450         1    1    1   -1    -1   -1   -1     4
451                         451         1    1    1   -1    -1   -1   -1     4
452                         452         1    1    1   -1    -1   -1   -1     4
453                         453         1    1    1   -1    -1   -1   -1     4
454                         454         1    1    1   -1    -1   -1   -1     4
455                         455         1    1    1   -1    -1   -1   -1     4
456                         456        11    3    5    

In [44]:
!ls /scratch/maghoi/pMHC_data/features6/|wc

   1464    1464   45384


In [45]:
!mkdir -p /scratch/maghoi/pMHC_data/features6/
!ls /scratch/maghoi/pMHC_data/features6/
!rm /scratch/maghoi/pMHC_data/features6/*

1g6r_1p_N1_1t_80s_pMHC-TCR.npy	3vxm_3p_P1_1t_80s_pMHC-TCR.npy
1g6r_1p_N1_1t_90s_pMHC-TCR.npy	3vxm_3p_P1_1t_90s_pMHC-TCR.npy
1g6r_1p_N1_1t_95s_pMHC-TCR.npy	3vxm_3p_P1_1t_95s_pMHC-TCR.npy
1g6r_1p_N1_1t_99s_pMHC-TCR.npy	3vxm_3p_P1_1t_99s_pMHC-TCR.npy
1g6r_1p_N2_1t_80s_pMHC-TCR.npy	3vxm_3p_R1_1t_80s_pMHC-TCR.npy
1g6r_1p_N2_1t_90s_pMHC-TCR.npy	3vxm_3p_R1_1t_90s_pMHC-TCR.npy
1g6r_1p_N2_1t_95s_pMHC-TCR.npy	3vxm_3p_R1_1t_95s_pMHC-TCR.npy
1g6r_1p_N2_1t_99s_pMHC-TCR.npy	3vxm_3p_R1_1t_99s_pMHC-TCR.npy
1g6r_1p_N3_1t_80s_pMHC-TCR.npy	3vxm_3p_S1_1t_80s_pMHC-TCR.npy
1g6r_1p_N3_1t_90s_pMHC-TCR.npy	3vxm_3p_S1_1t_90s_pMHC-TCR.npy
1g6r_1p_N3_1t_95s_pMHC-TCR.npy	3vxm_3p_S1_1t_95s_pMHC-TCR.npy
1g6r_1p_N3_1t_99s_pMHC-TCR.npy	3vxm_3p_S1_1t_99s_pMHC-TCR.npy
1g6r_1p_P1_1t_80s_pMHC-TCR.npy	3vxr_3p_N1_1t_80s_pMHC-TCR.npy
1g6r_1p_P1_1t_90s_pMHC-TCR.npy	3vxr_3p_N1_1t_90s_pMHC-TCR.npy
1g6r_1p_P1_1t_95s_pMHC-TCR.npy	3vxr_3p_N1_1t_95s_pMHC-TCR.npy
1g6r_1p_P1_1t_99s_pMHC-TCR.npy	3vxr_3p_N1_1t_99s_pMHC-TCR.npy
1g6r_1p_

In [46]:
for i in range(len(dfs)):
    filename = str(filelist_loaded[i].columns[0])
    np.save("/scratch/maghoi/pMHC_data/features6/" + filename, dfs[i])

In [47]:
!ls -al /scratch/maghoi/pMHC_data/features6/

total 304612
drwxr-xr-x  2 maghoi unixusers  73728 Mar 29 12:59 .
drwxr-xr-x 11 maghoi unixusers    295 Mar 28 21:19 ..
-rw-r--r--  1 maghoi unixusers 209792 Mar 29 12:59 1g6r_1p_N1_1t_80s_pMHC-TCR.npy
-rw-r--r--  1 maghoi unixusers 209792 Mar 29 12:59 1g6r_1p_N1_1t_90s_pMHC-TCR.npy
-rw-r--r--  1 maghoi unixusers 209792 Mar 29 12:59 1g6r_1p_N1_1t_95s_pMHC-TCR.npy
-rw-r--r--  1 maghoi unixusers 209792 Mar 29 12:59 1g6r_1p_N1_1t_99s_pMHC-TCR.npy
-rw-r--r--  1 maghoi unixusers 209792 Mar 29 12:59 1g6r_1p_N2_1t_80s_pMHC-TCR.npy
-rw-r--r--  1 maghoi unixusers 209792 Mar 29 12:59 1g6r_1p_N2_1t_90s_pMHC-TCR.npy
-rw-r--r--  1 maghoi unixusers 209792 Mar 29 12:59 1g6r_1p_N2_1t_95s_pMHC-TCR.npy
-rw-r--r--  1 maghoi unixusers 209792 Mar 29 12:59 1g6r_1p_N2_1t_99s_pMHC-TCR.npy
-rw-r--r--  1 maghoi unixusers 209792 Mar 29 12:59 1g6r_1p_N3_1t_80s_pMHC-TCR.npy
-rw-r--r--  1 maghoi unixusers 209792 Mar 29 12:59 1g6r_1p_N3_1t_90s_pMHC-TCR.npy
-rw-r--r--  1 maghoi unixusers 209792 Mar 29 12:59 1g6r_1p_N

In [48]:
np.load("/scratch/maghoi/pMHC_data/features6/5c0b_1p_N1_1t_95s_pMHC-TCR.npy")

array([[0.     , 0.     , 0.     , ..., 0.99907, 0.02968, 0.02968],
       [0.     , 0.     , 0.     , ..., 0.99211, 0.08627, 0.08627],
       [0.     , 0.     , 0.     , ..., 0.79091, 0.37219, 0.46524],
       ...,
       [0.     , 0.     , 0.     , ..., 0.05996, 0.01713, 0.01285],
       [0.     , 0.     , 0.     , ..., 0.00214, 0.02354, 0.02354],
       [0.     , 0.     , 0.     , ..., 0.16231, 0.01476, 0.01476]])

In [49]:
np.load("/scratch/maghoi/pMHC_data/features6/5men_0p_P1_1t_90s_pMHC-TCR.npy")

array([[0.     , 0.     , 0.     , ..., 0.99907, 0.02968, 0.02968],
       [0.     , 0.     , 0.     , ..., 0.99023, 0.10346, 0.08868],
       [0.     , 0.     , 0.     , ..., 0.70296, 0.43259, 0.54074],
       ...,
       [0.     , 0.     , 0.     , ..., 0.10244, 0.01707, 0.01067],
       [0.     , 0.     , 0.     , ..., 0.01498, 0.02354, 0.0214 ],
       [0.     , 0.     , 0.     , ..., 0.14996, 0.01479, 0.01267]])

Done!